# Análisis Exploratorio de las Reviews de Pitchfork

Este notebook realiza un análisis exploratorio de los datos almacenados en la base de datos PostgreSQL, con foco en las puntuaciones, géneros musicales y otros aspectos relevantes de las reviews.


In [ ]:
# Importación de librerías y carga de datos
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display
from sqlalchemy import create_engine
from dotenv import load_dotenv
from data_cleaning import clean_reviews
import os

sns.set(style="whitegrid")

load_dotenv()
DATABASE_URL = os.getenv('DATABASE_URL')
engine = create_engine(DATABASE_URL)

query = "SELECT * FROM pitchfork_reviews"
df = pd.read_sql(query, engine)
df.head()

print(f"Cantidad de registros: {len(df)}")



In [ ]:
# Limpieza de datos

df_clean = clean_reviews(df)
df_clean.head()

In [ ]:

# Distribución de Puntuaciones
plt.figure(figsize=(8, 6))
sns.histplot(df_clean['rating'], kde=True, bins=20)
plt.title('Distribución de Puntuaciones de Pitchfork')
plt.xlabel('Puntuación')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:

# Cantidad de Reviews por Género Musical
all_genres = df_clean['genre'].dropna().str.split(r'\s*/\s*')
genres_series = pd.Series([genre for sublist in all_genres for genre in sublist])
genre_count = genres_series.value_counts()

plt.figure(figsize=(12,6))
genre_count.plot(kind='bar')
plt.title('Cantidad de Reviews por Género Musical (Separados)')
plt.xlabel('Género')
plt.ylabel('Cantidad de Reviews')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Labels más recurrentes
label_count = df_clean['label'].value_counts().head(15)

plt.figure(figsize=(12,6))
label_count.plot(kind='bar')
plt.title('Top 15 Sellos Discográficos con más Reviews')
plt.xlabel('Sello')
plt.ylabel('Cantidad de Reviews')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Puntuación media por género (limpiando nulos)
genre_ratings = df_clean.dropna(subset=['genre', 'rating'])
genre_ratings.loc[:, 'genre_list'] = genre_ratings['genre'].str.split(r'\s*/\s*')
exploded = genre_ratings.explode('genre_list')
mean_rating_by_genre = exploded.groupby('genre_list')['rating'].mean().sort_values(ascending=False).head(15)

plt.figure(figsize=(12,6))
mean_rating_by_genre.plot(kind='bar', color='skyblue')
plt.title('Top 15 Géneros con Mejor Puntuación Media')
plt.xlabel('Género')
plt.ylabel('Puntuación Media')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Artistas mejor valorados (con mínimo 3 reviews)

best_artists_df = df.groupby('artist').filter(lambda x: len(x) >= 3)
best_artists_avg = best_artists_df.groupby('artist')['rating'].mean().sort_values(ascending=False).head(10).reset_index()
display(best_artists_avg)

In [ ]:
# Albums con puntuación perfecta
# Álbumnes con puntuación perfecta ordenados por año de lanzamiento
perfect_albums = df[df['rating'] == 10][['artist', 'title', 'release_year']].reset_index(drop=True)
perfect_albums = perfect_albums.sort_values(by='release_year', ascending=True).reset_index(drop=True)

display(perfect_albums.style.set_properties(**{'text-align': 'left'}))